## Gemini 기본 처리

### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet google-genai

### GCP 환경설정 및 로그인

In [2]:
import os

PROJECT_ID = "ai-hangsik" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}
USE_VERTEX_AI = True #@param {type:"boolean"}
MODEL = "gemini-2.5-flash" #@param {type:"string"}

In [ ]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project {PROJECT_ID}

### Gemini 실행

In [4]:
import base64
from IPython.display import Image, display, Markdown

from google import genai
from google.genai import types
from google.genai.types import HttpOptions

client = genai.Client(
    vertexai=USE_VERTEX_AI,
    project=PROJECT_ID,
    location=REGION,)

#### Text 입력

In [5]:
from google import genai
from google.genai.types import HttpOptions
from IPython.display import Image, display, Markdown


response = client.models.generate_content(
    model=MODEL,

    contents=[
      types.Content(
        role="user",
        parts=[
          types.Part.from_text(text="""트랜스포머 아키텍처에서 인코더란? """)
        ]
      )
    ],
)
display(Markdown(response.text))

트랜스포머(Transformer) 아키텍처에서 **인코더(Encoder)**는 주어진 **입력 시퀀스(Input Sequence)**를 이해하고, 그 의미를 **콘텍스트를 포함하는 고차원 벡터 표현(Contextualized High-Dimensional Vector Representation)**으로 변환하는 역할을 하는 부분입니다.

쉽게 비유하자면, 인코더는 복잡한 문장이나 데이터를 받아서 그 핵심적인 의미와 문맥 정보를 '해독'하고 '압축'하는 "이해의 전문가"라고 할 수 있습니다.

---

### 인코더의 주요 역할 및 특징

1.  **입력 시퀀스 이해 및 문맥 파악:**
    *   인코더의 핵심 목적은 입력된 단어들의 단순한 나열을 넘어서, 각 단어가 문맥 속에서 어떤 의미를 가지며 다른 단어들과 어떻게 상호작용하는지 깊이 있게 파악하는 것입니다.
    *   예를 들어, "사과를 깎다"의 '사과'와 "사과하세요"의 '사과'가 다른 의미임을 문맥을 통해 이해합니다.

2.  **병렬 처리 능력:**
    *   전통적인 순환 신경망(RNN)과 달리, 인코더는 입력 시퀀스의 모든 단어를 동시에 처리합니다. 이는 **셀프-어텐션(Self-Attention)** 메커니즘 덕분입니다. 덕분에 훨씬 빠른 학습과 효율적인 처리가 가능합니다.

3.  **주요 구성 요소:**
    *   트랜스포머 인코더는 여러 개의 동일한 **인코더 레이어(Encoder Layer)**를 쌓아 올린 구조입니다. 각 인코더 레이어는 다음 두 가지 핵심 서브 레이어로 구성됩니다:
        *   **멀티-헤드 셀프-어텐션(Multi-Head Self-Attention) 메커니즘**: 입력 시퀀스의 각 단어가 시퀀스 내의 다른 모든 단어들과 얼마나 관련이 있는지 파악하여, 단어의 의미를 문맥에 따라 조정합니다. 여러 개의 "헤드"를 통해 다양한 관점에서 관계를 파악합니다.
        *   **피드-포워드 신경망(Feed-Forward Neural Network)**: 어텐션 결과로 나온 문맥 정보를 각 단어별로 독립적으로 처리하여 더 높은 수준의 추상화된 특징을 추출합니다.
    *   **위치 인코딩(Positional Encoding)**: 단어 자체의 임베딩 외에, 단어들의 순서 정보를 제공하기 위해 추가됩니다. 트랜스포머는 순환 구조가 없기 때문에 이 위치 정보가 필수적입니다.
    *   **잔차 연결(Residual Connections)** 및 **레이어 정규화(Layer Normalization)**: 안정적인 학습과 깊은 네트워크에서도 정보가 잘 흐르도록 돕습니다.

4.  **출력:**
    *   인코더의 최종 출력은 입력 시퀀스의 각 단어에 대한 **문맥 정보가 풍부한 벡터 표현들의 시퀀스**입니다. 이 벡터들은 이후 디코더(Encoder-Decoder 모델의 경우)로 전달되거나, 인코더-온리 모델에서는 직접적으로 다음 태스크(분류, 개체명 인식 등)에 활용됩니다.

---

### 인코더의 활용 예시

*   **인코더-디코더 모델 (예: 기계 번역)**:
    *   소스 언어(예: 한국어) 문장을 인코더가 입력받아 이해하고, 그 문맥 정보를 담은 벡터를 생성합니다. 이 벡터는 디코더(Decoder)가 타겟 언어(예: 영어) 문장을 생성하는 데 필요한 '원천 정보' 역할을 합니다. 즉, "이런 의미의 한국어 문장이 들어왔으니, 이제 이걸 영어로 번역해!"라고 디코더에게 알려주는 것입니다.

*   **인코더-온리 모델 (예: BERT, RoBERTa)**:
    *   BERT와 같은 모델들은 인코더만을 사용합니다. 이 경우 인코더 자체로 입력 텍스트의 문맥적 이해를 심화시키고, 그 이해를 바탕으로 다양한 자연어 처리 태스크(감성 분석, 질의응답, 문장 분류 등)를 수행합니다. 디코더 없이 인코더의 최종 출력을 바로 특정 태스크에 맞춰 변형하여 사용합니다.

결론적으로, 트랜스포머 아키텍처에서 인코더는 **입력 데이터의 심층적인 문맥적 이해를 담당하는 핵심 구성 요소**이며, 특히 셀프-어텐션을 통해 단어 간의 복잡한 관계와 장거리 의존성을 효과적으로 학습하는 능력이 뛰어납니다.

#### Image 입력

In [6]:
img_file_uri="https://storage.googleapis.com/cloud-samples-data/generative-ai/image/scones.jpg"
display(Image(url=img_file_uri, width=400))

In [7]:
from google import genai
from google.genai.types import HttpOptions, Part

response = client.models.generate_content(
    model=MODEL,
    contents=[
        Part.from_text(text="""이미지에 보여지는 내용을 설명해주세요"""),
        Part.from_uri(
            file_uri= img_file_uri,
            mime_type="image/jpeg",
        ),
    ],
)
display(Markdown(response.text))

이 이미지는 따뜻하고 아늑한 분위기의 상차림을 위에서 내려다본 모습입니다.

주요 내용은 다음과 같습니다:

1.  **블루베리 스콘**: 하얀 유산지 위에 갓 구운 듯한 둥근 블루베리 스콘 여러 개가 놓여 있습니다. 스콘 위에는 블루베리 조각들과 설탕 알갱이가 보이며, 유산지에는 블루베리 색상으로 보이는 보라색 얼룩들이 자연스럽게 묻어 있습니다.
2.  **음료**:
    *   왼쪽 상단에는 진한 검은색 커피가 담긴 하얀 컵이 있습니다.
    *   오른쪽 하단에는 우유가 들어간 듯한 밝은 색 커피가 담긴 또 다른 하얀 컵이 보입니다.
3.  **블루베리**: 스콘 주변과 유산지 위에는 신선한 블루베리들이 흩어져 있으며, 작은 어두운 색 그릇에도 블루베리가 가득 담겨 있습니다.
4.  **소품**: 블루베리 그릇 옆에는 'LET'S JAM'이라고 새겨진 은색 숟가락이 놓여 있습니다. 또한 스콘 근처에는 싱그러운 민트 잎 하나가 놓여 있어 색다른 포인트를 줍니다.
5.  **꽃**: 이미지의 오른쪽에는 푸른 줄기와 잎을 가진 크고 분홍색의 작약(모란)으로 보이는 꽃들이 아름답게 배치되어 있습니다. 몇몇 꽃잎이 떨어져 있어 자연스러운 느낌을 더합니다.
6.  **배경**: 전체적인 배경은 청록색과 갈색이 섞인 질감 있는 표면으로, 빈티지하거나 소박한 느낌을 줍니다. 유산지 아래로는 잡지나 다른 종이들이 살짝 비치는 모습도 보입니다.

전체적으로 잘 차려진 아침 식사 또는 브런치 테이블의 모습으로, 신선한 재료와 예쁜 꽃이 어우러져 편안하고 감성적인 분위기를 자아냅니다.

#### Youtube 분석

In [8]:
from google import genai
from google.genai.types import HttpOptions, Part

response = client.models.generate_content(
    model=MODEL,
    contents=[
        Part.from_uri(
            file_uri="https://www.youtube.com/watch?v=3KtWfp0UopM",
            mime_type="video/mp4",
        ),
        Part.from_text(text="""이 영상을 바탕으로 짧고 매력적인 블로그 게시물을 한국어로 작성해 보세요."""),
    ],
)

display(Markdown(response.text))

## 구글 창립 25주년: 검색으로 되짚어본 '가장 많이 찾아본' 우리의 발자취

구글이 25번째 생일을 맞이했습니다! 지난 25년간 전 세계인의 궁금증을 해소하고 지식을 연결하는 창구 역할을 해왔죠. 구글이 25주년을 기념해 공개한 특별 영상은 우리가 무엇을 '가장 많이 검색'했는지를 보여주며, 인류의 집단 지성과 문화의 흐름을 한눈에 볼 수 있게 합니다.

### 호기심으로 시작된 탐험의 역사

영상은 문명사의 위대한 흔적(이집트 피라미드)부터 인류의 가장 큰 발걸음(닐 암스트롱의 달 착륙), 그리고 최초의 여성 우주비행사의 우주여행까지, 역사의 중요한 순간들이 어떻게 검색을 통해 다시 소환되고 기억되는지를 보여줍니다. 이 모든 것은 인류의 끊임없는 호기심이 시대를 관통하며 검색을 이끌어왔음을 상기시켜 줍니다.

### 대중문화의 중심, 검색 트렌드

문화적 아이콘들 역시 검색 트렌드의 중요한 부분입니다. 1980년대 패션과 댄스, 전 세계를 K-POP 열풍으로 물들인 방탄소년단(BTS)과 블랙핑크(BLACKPINK), 매혹적인 발리우드 영화, 그리고 올 한 해를 뜨겁게 달군 바비 인형까지. 검색은 단순한 정보 획득을 넘어, 시대의 감성과 트렌드를 공유하는 창이 됩니다.

### 공감하고 행동하는 인류의 기록

스포츠 스타 크리스티아누 호날두와 르브론 제임스처럼 경계를 허무는 영웅들의 활약과 함께, 재난 현장에서 '어떻게 도울까?'를 검색하며 보여주는 따뜻한 인류애, 그리고 인종차별에 맞서는 'Black Lives Matter' 운동과 같은 사회적 움직임도 검색의 기록으로 남습니다. 검색은 우리가 단순히 질문하는 존재를 넘어, 공감하고 행동하는 존재임을 증명합니다.

### 미래를 향한 무한한 가능성

영상 말미에는 테니스 선수 코코 가우프처럼 새로운 역사를 쓰는 이들, 그리고 루빅스 큐브를 맞추고, 스케이트보드를 타고, 드럼을 연주하며 자신의 가능성을 탐색하는 어린 세대의 모습이 담겨 있습니다. 이들은 끊임없이 기준을 높이고, "아무것도 불가능하지 않다"는 메시지를 전하며 미래를 향한 희망을 보여줍니다.

이 25년간의 검색 데이터는 단순한 디지털 기록이 아닙니다. 인류의 지적 호기심과 문화적 변화, 그리고 공감하는 사회적 의식을 담아낸 살아있는 역사책이죠. 앞으로도 구글과 함께 더 많은 것을 찾고, 더 많은 것을 배우며, 더 나은 미래를 만들어가길 기대합니다. "Search on!"

## End of Document